In [1]:
!nvcc --version  # correct double dash


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-nnyujwr3
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-nnyujwr3
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-bo6jvgnk/wheels/ef/1d/c6/f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter


Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpovpbgvyo".


In [4]:
code = r'''
#include <stdio.h>
#include <cuda.h>

__global__ void maxReduce(int* input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int num_threads = blockDim.x;

    while (num_threads > 0) {
        if (tid < num_threads && (tid * step_size * 2 + step_size) < blockDim.x * 2) {
            int fst = tid * step_size * 2;
            int snd = fst + step_size;
            if (input[fst] < input[snd]) {
                input[fst] = input[snd];
            }
        }
        __syncthreads();
        step_size <<= 1;
        num_threads >>= 1;
    }
}

int main() {
    const int count = 8;
    const int size = count * sizeof(int);
    int h[count] = {13, 65, 15, 14, 33, 2, 30, 8};

    int* d;
    cudaMalloc(&d, size);
    cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);

    maxReduce<<<1, count / 2>>>(d);

    int result;
    cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);
    printf("Largest number is at index: %d\n", result);

    cudaFree(d);
    return 0;
}
'''

with open('max_reduce.cu', 'w') as f:
    f.write(code)


In [5]:
!nvcc max_reduce.cu -o max_reduce

In [6]:
!./max_reduce

Largest number is at index: 4
